#### Project Objective:

Analyse the Pokémon dataset to determine which Pokémon types are best suited for battles based on aggregated combat statistics, and provide recommendations on the optimal team composition for competitive play.

In [0]:
# Load the Pokémon dataset from the uploaded CSV file in DBFS
pokemon_df = spark.read.option("header", "true").csv("dbfs:/FileStore/shared_uploads/fifthfrankie@gmail.com/Pokemon.csv")
pokemon_df.show(5)

+---+--------------------+------+------+-----+---+------+-------+-------+-------+-----+----------+---------+
|  #|                Name|Type 1|Type 2|Total| HP|Attack|Defense|Sp. Atk|Sp. Def|Speed|Generation|Legendary|
+---+--------------------+------+------+-----+---+------+-------+-------+-------+-----+----------+---------+
|  1|           Bulbasaur| Grass|Poison|  318| 45|    49|     49|     65|     65|   45|         1|    False|
|  2|             Ivysaur| Grass|Poison|  405| 60|    62|     63|     80|     80|   60|         1|    False|
|  3|            Venusaur| Grass|Poison|  525| 80|    82|     83|    100|    100|   80|         1|    False|
|  3|VenusaurMega Venu...| Grass|Poison|  625| 80|   100|    123|    122|    120|   80|         1|    False|
|  4|          Charmander|  Fire|  null|  309| 39|    52|     43|     60|     50|   65|         1|    False|
+---+--------------------+------+------+-----+---+------+-------+-------+-------+-----+----------+---------+
only showing top 5 

In [0]:
from pyspark.sql.functions import col

# Rename columns to remove spaces and dots for easier handling in Spark SQL
pokemon_df = pokemon_df.withColumnRenamed("Sp. Atk", "Sp_Atk")
pokemon_df = pokemon_df.withColumnRenamed("Sp. Def", "Sp_Def")
pokemon_df = pokemon_df.withColumnRenamed("Type 1", "Type_1")
pokemon_df = pokemon_df.withColumnRenamed("Type 2", "Type_2")

# Convert numerical fields from string to integer
numerical_cols = ['Total', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed']
for col_name in numerical_cols:
    pokemon_df = pokemon_df.withColumn(col_name, col(col_name).cast("integer"))

# Handle missing values
pokemon_df = pokemon_df.na.fill(value=0)

# Show the schema to confirm changes
pokemon_df.printSchema()
pokemon_df.describe().show()

root
 |-- #: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type_1: string (nullable = true)
 |-- Type_2: string (nullable = true)
 |-- Total: integer (nullable = true)
 |-- HP: integer (nullable = true)
 |-- Attack: integer (nullable = true)
 |-- Defense: integer (nullable = true)
 |-- Sp_Atk: integer (nullable = true)
 |-- Sp_Def: integer (nullable = true)
 |-- Speed: integer (nullable = true)
 |-- Generation: string (nullable = true)
 |-- Legendary: string (nullable = true)

+-------+------------------+----------------+------+------+------------------+------------------+-----------------+------------------+----------------+-----------------+------------------+------------------+---------+
|summary|                 #|            Name|Type_1|Type_2|             Total|                HP|           Attack|           Defense|          Sp_Atk|           Sp_Def|             Speed|        Generation|Legendary|
+-------+------------------+----------------+------+------+---

In [0]:
from pyspark.sql.functions import when, col

# Calculate Combat Power using renamed and cleaned numerical columns
pokemon_df = pokemon_df.withColumn(
    "Combat_Power",
    col("HP") + col("Attack") + col("Defense") + col("Sp_Atk") + col("Sp_Def") + col("Speed")
)

# Classify Pokémon into categories based on Combat Power
pokemon_df = pokemon_df.withColumn(
    "Power_Category",
    when(col("Combat_Power") > 500, "High")
    .when(col("Combat_Power") > 300, "Medium")
    .otherwise("Low")
)

# Calculate average Combat Power grouped by Type_1
avg_combat_power_df = pokemon_df.groupBy("Type_1").avg("Combat_Power").withColumnRenamed("avg(Combat_Power)", "Avg_Combat_Power")
avg_combat_power_df.show()

+--------+------------------+
|  Type_1|  Avg_Combat_Power|
+--------+------------------+
|   Water|430.45535714285717|
|  Poison|399.14285714285717|
|   Steel| 487.7037037037037|
|    Rock|            453.75|
|     Ice| 433.4583333333333|
|   Ghost|          439.5625|
|   Fairy| 413.1764705882353|
| Psychic|475.94736842105266|
|  Dragon|         550.53125|
|  Flying|             485.0|
|     Bug|378.92753623188406|
|Electric|443.40909090909093|
|    Fire| 458.0769230769231|
|  Ground|             437.5|
|    Dark|  445.741935483871|
|Fighting|416.44444444444446|
|   Grass|421.14285714285717|
|  Normal|401.68367346938777|
+--------+------------------+



In [0]:
# Store the transformed dataset in Delta Lake
pokemon_df.write.format("delta").mode("overwrite").save("/mnt/delta/pokemon_data")

In [0]:
# Analyse - Fetch the top 5 Pokémon Types based on average Combat Power
top_types_df = avg_combat_power_df.orderBy(col("Avg_Combat_Power").desc()).limit(5)
top_types_df.show()

# Visualisation - Display the average Combat Power per Type using Databricks' display function
display(top_types_df)

+-------+------------------+
| Type_1|  Avg_Combat_Power|
+-------+------------------+
| Dragon|         550.53125|
|  Steel| 487.7037037037037|
| Flying|             485.0|
|Psychic|475.94736842105266|
|   Fire| 458.0769230769231|
+-------+------------------+



Type_1,Avg_Combat_Power
Dragon,550.53125
Steel,487.7037037037037
Flying,485.0
Psychic,475.94736842105266
Fire,458.0769230769231
